# 第3章：Strands Agents中的自定义工具

## 自定义工具简介

虽然内置工具为许多任务提供了坚实的基础，但Strands Agents的真正力量来自于能够创建适合您特定需求的自定义工具。在本章中，我们将探讨如何创建、完善和扩展您自己的工具，以增强代理的能力。

自定义工具允许您：

- 将代理连接到您自己的API和服务

- 创建特定领域的功能

- 构建专门的数据处理工具

- 与数据库和外部系统对接

- 实现特定于您的用例的业务逻辑

在本章中，我们将按照课程要求使用Claude 3.7 Sonnet模型（`us.anthropic.claude-3-7-sonnet-20250219-v1:0`）。

## 设置和前提条件

让我们从安装必要的包开始：

In [1]:
# Install strands-agents and strands-agents-tools if you haven't already
%pip install -U strands-agents strands-agents-tools

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Import the AWS SDK for Python
import boto3
import os

# Option 1: Set environment variables (uncomment and set your values)
os.environ['AWS_ACCESS_KEY_ID'] = '...'
os.environ['AWS_SECRET_ACCESS_KEY'] = '...'
os.environ['AWS_REGION'] = '...'
# Option 2: Create a boto3 session with your credentials
# session = boto3.Session(
#     aws_access_key_id='your_access_key_id',
#     aws_secret_access_key='your_secret_access_key',
#     region_name='us-west-2'  # or your preferred region
# )

# Verify your configuration
try:
    boto3.client('bedrock-runtime')
    print("AWS credentials configured correctly!")
except Exception as e:
    print(f"Error configuring AWS credentials: {e}")
    print("Please set your AWS credentials before proceeding.")

AWS credentials configured correctly!


## 创建你的第一个自定义Tool

在Strands Agents中创建自定义工具最简单的方法是使用`@tool`装饰器与Python函数：

In [3]:
from strands import Agent, tool

# Define a simple custom tool
@tool
def word_counter(text: str) -> dict:
    """
    统计文本中每个单词的出现次数。
参数：
text (str)：要分析的输入文本
返回：
dict：一个将单词映射到其出现次数的字典
    """
    # Remove punctuation and convert to lowercase
    import string
    text = text.lower()
    for punct in string.punctuation:
        text = text.replace(punct, ' ')
    
    # Split into words and count occurrences
    words = text.split()
    word_counts = {}
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1
    
    return word_counts

# Create an agent with our custom tool
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[word_counter],
    system_prompt="你是一个能够分析文本并提供词语统计的助手。"
)

Let's test our custom tool by asking the agent to analyze some text:

In [4]:
response = agent("""
请分析以下段落并告诉我哪些词出现频率最高：
"Strands Agents是一个使用Python构建AI代理的强大框架。
通过Strands Agents，你可以创建利用Nova Pro等语言模型
来执行复杂任务的智能代理。该框架提供了增强
你的代理各种能力的工具，使构建复杂的AI
应用变得简单。"
""")

我可以帮你分析这段文本并找出出现频率最高的词。让我使用word_counter工具来统计每个词的出现次数。
Tool #1: word_counter
我注意到词语统计结果似乎将整个文本分成了几个大块，而不是按照单个词进行统计。这可能是因为中文文本的分词处理方式不同。

让我重新尝试，将文本中的英文部分与中文分开处理：
Tool #2: word_counter
分析结果显示，在这段文本中出现频率最高的词语是：

1. "的" - 出现了5次
2. "代理" - 出现了3次
5. "。" (句号) - 出现了3次
3. "Strands" - 出现了2次
4. "Agents" - 出现了2次
5. "构建" - 出现了2次
6. "AI" - 出现了2次
7. "框架" - 出现了2次
8. "，"(逗号) - 出现了2次
9. "你" - 出现了2次
10. "复杂" - 出现了2次

如果我们忽略标点符号，那么"的"和"代理"是这段文本中出现最频繁的词语。这反映了这段文本主要讨论的是关于构建AI代理的内容。

## 自定义工具的解析
让我们分解Strands Agents中自定义工具的关键组成部分：
1. **`@tool`装饰器**：这告诉Strands该函数应被视为工具
2. **Type注解**：用于定义预期的输入和输出类型
3. **文档字符串**：对于代理理解何时以及如何使用工具至关重要
4. **实现**：工具被使用时实际执行的代码

### 良好文档字符串的重要性

文档字符串特别重要，因为它帮助AI模型理解：

- 工具的功能
- 何时使用它
- 它期望的输入
- 它产生的输出


In [5]:
@tool
def text_sentiment_analyzer(text: str) -> dict:
    """
    使用简单的基于词典的方法分析文本情感。
这个工具通过计算预定义词典中的积极和消极词汇来对文本情感进行分类。它适用于快速评估用户反馈、产品评论或社交媒体评论的情感。

参数：
text (str)：要分析情感的文本。应该是英文，最好包含至少10个单词以获得更好的准确性。

返回：
dict：包含以下内容的字典：
- 'sentiment'：整体情感（'positive'、'negative'或'neutral'）
- 'score'：情感分数，从-1.0（非常消极）到1.0（非常积极）
- 'positive_words'：文本中发现的积极词汇列表
- 'negative_words'：文本中发现的消极词汇列表

注意：这是一个用于演示目的的简化情感分析器。
    """
    # Simple lexicons
    positive_words = {
        'good', 'great', 'excellent', 'wonderful', 'fantastic', 'amazing', 'love', 'best',
        'happy', 'joy', 'positive', 'helpful', 'beautiful', 'perfect', 'recommend'
    }
    
    negative_words = {
        'bad', 'terrible', 'awful', 'horrible', 'poor', 'worst', 'hate', 'dislike', 
        'negative', 'disappointed', 'disappointing', 'useless', 'waste', 'broken'
    }
    
    # Normalize text
    text = text.lower()
    words = ''.join(c if c.isalnum() else ' ' for c in text).split()
    
    # Find positive and negative words
    found_positive_words = [word for word in words if word in positive_words]
    found_negative_words = [word for word in words if word in negative_words]
    
    # Calculate sentiment score
    pos_count = len(found_positive_words)
    neg_count = len(found_negative_words)
    total_count = pos_count + neg_count
    
    if total_count == 0:
        sentiment = "neutral"
        score = 0.0
    else:
        score = (pos_count - neg_count) / total_count
        if score > 0.1:
            sentiment = "positive"
        elif score < -0.1:
            sentiment = "negative"
        else:
            sentiment = "neutral"
    
    # Return results
    return {
        'sentiment': sentiment,
        'score': round(score, 2),
        'positive_words': found_positive_words,
        'negative_words': found_negative_words
    }

现在，让我们创建一个同时具有我们两个自定义工具的代理，并测试它如何使用它们：

In [7]:
text_analysis_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[word_counter, text_sentiment_analyzer],
    system_prompt="You are an assistant that specializes in text analysis."
)

response = text_analysis_agent("""
请分析这两篇产品评论，告诉我它们的情感和语言使用情况：

评论1："我绝对喜欢这个产品！它使用简单，运行完美。设计很漂亮，这是我今年做出的最佳购买。"

评论2："这真是令人失望。产品到达时已经损坏，客服也毫无帮助。省下你的钱，避开这个糟糕的产品。"
""")

我可以帮您分析这两篇产品评论的情感和语言使用情况。我将使用工具来分析英文文本的情感和词汇使用情况，因此需要先将这些评论翻译成英文。

首先，让我分析第一篇评论：
Tool #3: text_sentiment_analyzer

Tool #4: word_counter
现在分析第二篇评论：
Tool #5: text_sentiment_analyzer

Tool #6: word_counter
### 评论分析结果

**评论1分析：**
- **情感分析**：非常积极正面 (得分 1.0)
- **积极词汇**：love (喜爱)、beautiful (漂亮)、best (最佳)
- **没有负面词汇**
- **语言特点**：
  - 使用了强烈的肯定性词语，如"绝对喜欢"、"完美"、"最佳"
  - 突出产品的多个优点：易用性、性能、设计美观
  - 表达了个人满意度和推荐意愿
  - 用词积极、热情

**评论2分析：**
- **情感分析**：非常消极负面 (得分 -1.0)
- **负面词汇**：disappointing (令人失望)、terrible (糟糕)
- **没有积极词汇**
- **语言特点**：
  - 使用了明确的负面评价词，如"失望"、"糟糕"
  - 指出了具体问题：产品损坏、客服不佳
  - 包含警告和劝阻其他消费者的语言
  - 用词直接、批评性强

**总结对比**：
这两篇评论代表了产品评价的两个极端。第一篇充满热情和赞美，表达了高度满意；第二篇则表达了强烈的不满和失望，甚至试图劝阻他人购买。这种鲜明对比展示了客户体验可能的巨大差异，对于了解产品优缺点和改进方向具有重要参考价值。

## Tools输入和输出类型

Strands支持各种工具的输入和输出类型。让我们探索一些常见模式：

In [8]:
@tool
def text_formatter(text: str, format_type: str, max_length: int = 100) -> str:
    """
    根据指定的格式选项格式化文本。\
参数：
text (str)：要格式化的输入文本
format_type (str)：要应用的格式化类型。有效选项有：
'uppercase'、'lowercase'、'title_case'、'sentence_case'、'truncate'
max_length (int, 可选)：使用'truncate'格式时的最大长度。默认为100。

返回：
str：格式化后的文本
    """
    format_type = format_type.lower()
    
    if format_type == 'uppercase':
        return text.upper()
    elif format_type == 'lowercase':
        return text.lower()
    elif format_type == 'title_case':
        return text.title()
    elif format_type == 'sentence_case':
        return '. '.join(s.capitalize() for s in text.split('. '))
    elif format_type == 'truncate':
        if len(text) <= max_length:
            return text
        return text[:max_length] + '...'
    else:
        return f"Error: Unknown format type '{format_type}'."

### 处理复杂数据类型

In [9]:
from typing import List, Dict, Any

@tool
def data_summarizer(data: List[Dict[str, Any]], fields_to_summarize: List[str]) -> Dict[str, Any]:
    """
    在字典列表（例如记录或JSON数据）中汇总数值字段。
参数：
data (List[Dict])：要汇总的字典（记录）列表
fields_to_summarize (List[str])：要包含在汇总中的字段名称列表

返回：
Dict：包含每个指定字段汇总统计的字典：
- count：记录数量
- min：最小值
- max：最大值
- avg：平均值
- sum：值的总和
    """
    result = {}
    
    # Check if there's any data to summarize
    if not data:
        return {"error": "未提供数据"}
    
    # Process each requested field
    for field in fields_to_summarize:
        # Check if the field exists in the records
        valid_values = []
        for record in data:
            if field in record and isinstance(record[field], (int, float)):
                valid_values.append(record[field])
        
        # Generate statistics if we have valid values
        if valid_values:
            result[field] = {
                "count": len(valid_values),
                "min": min(valid_values),
                "max": max(valid_values),
                "avg": sum(valid_values) / len(valid_values),
                "sum": sum(valid_values)
            }
        else:
            result[field] = {"error": "未找到有效值"}
    
    return result

让我们用我们的新工具创建一个代理并测试它们：

In [10]:
advanced_tools_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[text_formatter, data_summarizer],
    system_prompt="You are an assistant with advanced text and data processing capabilities."
)

response = advanced_tools_agent("""
我有几个任务给你：
1. 将以下文本格式化为标题格式：
"strands agents是一个强大的框架，用于构建具有高级功能的AI助手。"
2. 总结这些销售数据：

[
{"product": "笔记本电脑", "price": 1200, "units_sold": 45, "rating": 4.7},
{"product": "手机", "price": 800, "units_sold": 125, "rating": 4.5},
{"product": "平板电脑", "price": 350, "units_sold": 85, "rating": 4.2},
{"product": "耳机", "price": 200, "units_sold": 155, "rating": 4.8},
{"product": "显示器", "price": 400, "units_sold": 60, "rating": 4.6}
]

对于销售数据，我想要关于price、units_sold和rating字段的统计数据。
""")

我将帮您完成这两个任务：

### 任务1：将文本格式化为标题格式

我会使用text_formatter工具将您提供的文本转换为标题格式（title_case）。
Tool #7: text_formatter
### 任务2：总结销售数据

我将使用data_summarizer工具来分析您提供的销售数据，并统计price、units_sold和rating字段。
Tool #8: data_summarizer
## 结果总结：

1. **文本格式化结果**：
   "Strands Agents是一个强大的框架，用于构建具有高级功能的Ai助手。"

2. **销售数据统计**：

   **价格(price)统计**:
   - 记录数量: 5
   - 最低价格: 200
   - 最高价格: 1200
   - 平均价格: 590.0
   - 总价值: 2950

   **销售数量(units_sold)统计**:
   - 记录数量: 5
   - 最低销售量: 45
   - 最高销售量: 155
   - 平均销售量: 94.0
   - 总销售量: 470

   **评分(rating)统计**:
   - 记录数量: 5
   - 最低评分: 4.2
   - 最高评分: 4.8
   - 平均评分: 4.56
   - 总评分: 22.8

## 为自定义工具添加错误处理

为生产级工具添加健全的错误处理至关重要。以下是一个具有适当错误处理的示例：

In [12]:
@tool
def robust_text_analyzer(text: str, analysis_type: str) -> Dict[str, Any]:
    """
    使用不同的分析方法分析文本，具有强大的错误处理功能。
参数：
text (str)：要分析的文本
analysis_type (str)：要执行的分析类型。有效选项有：
'word_count'、'char_count'、'sentence_count'、'readability'

返回：
dict：分析结果，如果出现错误则返回错误信息
    """
    try:
        # Input validation
        if not isinstance(text, str):
            return {"error": "Input text must be a string"}
        
        if not text.strip():
            return {"error": "Input text is empty"}
        
        analysis_type = analysis_type.lower()  # Normalize input
        valid_types = {'word_count', 'char_count', 'sentence_count', 'readability'}
        
        if analysis_type not in valid_types:
            return {
                "error": f"Invalid analysis type: '{analysis_type}'.", 
                "valid_options": list(valid_types)
            }
        
        # Perform the requested analysis
        if analysis_type == 'word_count':
            words = text.split()
            return {
                "total_words": len(words),
                "unique_words": len(set(words))
            }
            
        elif analysis_type == 'char_count':
            return {
                "total_chars": len(text),
                "letters": sum(c.isalpha() for c in text),
                "digits": sum(c.isdigit() for c in text),
                "spaces": sum(c.isspace() for c in text)
            }
            
        elif analysis_type == 'sentence_count':
            import re
            sentences = re.split(r'[.!?](?:\s|$)', text)
            sentences = [s for s in sentences if s.strip()]
            return {
                "sentence_count": len(sentences),
                "avg_sentence_length": len(text) / max(len(sentences), 1)
            }
            
        elif analysis_type == 'readability':
            words = text.split()
            if not words:
                return {"error": "Cannot calculate readability for empty text"}
                
            avg_word_length = sum(len(word) for word in words) / len(words)
            
            if avg_word_length < 4:
                difficulty = "Easy"
            elif avg_word_length < 6:
                difficulty = "Medium"
            else:
                difficulty = "Complex"
                
            return {
                "avg_word_length": avg_word_length,
                "difficulty_estimate": difficulty
            }
    
    except Exception as e:
        # Catch and report any unexpected errors
        return {"error": f"Analysis failed with error: {str(e)}"}

# Create an agent with our robust tool
robust_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[robust_text_analyzer],
    system_prompt="You are an assistant that can analyze text with high reliability."
)

## 需要外部依赖的Tools

自定义工具可以利用外部库提供高级功能。以下是一个使用NLTK进行文本处理的简化示例：

In [15]:
# This is an example - you'd need to install NLTK first with:
!pip install nltk

@tool
def nlp_tool(text: str, operation: str) -> Dict[str, Any]:
    """
    使用NLP技术处理文本。
参数：
text: 要处理的文本
operation: 要执行的NLP操作（'tokenize'、'pos_tag'、'entities'）

返回：
包含NLP处理结果的字典
    """
    try:
        import nltk
        # You would need to download nltk data packages first:
        # nltk.download('punkt')
        # nltk.download('averaged_perceptron_tagger')
        
        if operation == 'tokenize':
            return {
                "tokens": nltk.word_tokenize(text),
                "sentences": nltk.sent_tokenize(text)
            }
        elif operation == 'pos_tag':
            tokens = nltk.word_tokenize(text)
            return {"tagged": nltk.pos_tag(tokens)}
        else:
            return {"error": f"Unknown operation: {operation}"}
    except Exception as e:
        return {"error": str(e)}

# 注意：这只是一个例子，需要安装NLTK

## 自定义工具的最佳实践

创建Strands Agents的自定义工具时，请记住以下最佳实践：

1. **编写清晰的文档字符串**：文档字符串是模型理解您工具用途和参数的方式。要详细且明确。
2. **使用类型注解**：类型提示帮助模型理解您的工具期望的数据类型和返回值。
3. **优雅地处理错误**：返回信息丰富的错误消息，而不是让异常传播。
4. **保持工具专注**：每个工具应该把一件事做好，而不是尝试处理多个不相关的任务。
5. **提供输入验证**：尽早验证输入，以防止处理无效数据。
6. **使用描述性名称**：选择能清晰传达其用途的函数和参数名称。
7. **返回结构化数据**：尽可能返回结构化数据（字典、列表），使模型易于处理。
8. **考虑性能**：优化可能频繁调用或处理大量数据的工具。
9. **彻底测试**：使用各种输入（包括边缘情况）测试您的工具，确保它们按预期运行。
10. **版本依赖关系**：如果您的工具依赖外部库，请指定版本要求。

## 摘要

在本章中，我们探讨了：
- 使用`@tool`装饰器创建自定义工具
- 清晰文档字符串和类型注解的重要性
- 使用各种输入和输出类型
- 为工具添加强大的错误处理
- 将外部库集成到自定义工具中
- 工具开发的最佳实践
自定义工具是Strands Agents最强大的功能之一，允许您几乎向任何方向扩展代理的能力。通过创建设计良好的工具，您可以使您的代理执行复杂的、特定领域的任务，这些任务远远超出简单的文本生成。
在下一章中，我们将探讨如何将Strands Agents与模型上下文协议(MCP)集成，这使得更强大的工具集成和互操作性成为可能。

## 练习

1. 创建一个自定义工具，可以加载和解析CSV文件，然后对数据执行基本的数据分析操作。
2. 构建一个连接天气API并返回给定位置天气预报的工具。
3. 开发一个使用Pillow等库执行图像操作（如调整大小、格式转换）的工具。
4. 创建一个与数据库交互以存储和检索信息的自定义工具。
5. 设计一个验证和格式化JSON或XML等结构化数据的工具。